In [12]:
from agents import Agent, Runner, OpenAIChatCompletionsModel, function_tool
from dotenv import load_dotenv
from openai import AsyncOpenAI
import os
import requests
import nest_asyncio
import asyncio

In [13]:
load_dotenv(override=True)
nest_asyncio.apply()

In [7]:
GOOGLE_API_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
google_api_client = AsyncOpenAI(base_url=GOOGLE_API_URL, api_key=GOOGLE_API_KEY)
google_api_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=google_api_client)

In [9]:
@function_tool
def get_logs():
    """
    Fetches logs from the local server running at http://localhost:30007/logs.

    Returns:
        str: The logs retrieved from the server as plain text.
    """
    response = requests.get("http://localhost:30007/logs")
    return response.text

In [18]:
instructions_for_support_engineer = "You are a Support engineer, very keen in observing issues, \
reading logs and giving the right fix, you can get logs form the get_logs tool to get the logs, \
ignore the warnings and concentrate only on the errors, \
and also check for logs in sequential manner, do not give same error different solutions, read whole of the log"
# + "you use the file writing tool to paste the a solution in it,"

In [19]:
agent = Agent(
            name="Assistant", 
            instructions="You are a helpful assistant", 
            model=google_api_model,
            tools=[get_logs]
        )

result = await Runner.run(agent, "get the logs and give solution if any errors are seen")
print(result.final_output)

Okay, I have reviewed the logs. Here's what I found:

*   **Warning:** The server is running in development mode. This is not suitable for production environments.
*   **404 Error:** A request to `/fail` resulted in a 404 Not Found error. This means the requested resource does not exist on the server.
*   **500 Error:** A request to `/feature_1` resulted in a 500 Internal Server Error. This indicates a server-side issue. Looking at the traceback, it seems the error originates from a division by zero (`ZeroDivisionError`) within the `fail` function, which then raises a `Simulated error for testing` exception.

**Solutions:**

*   **404 Error:** If the `/fail` route is intended to exist, you need to define it in your application code (e.g., using `@app.route('/fail')` in Flask). If it's not meant to be a valid route, then the client (whoever is sending the request) should stop requesting it.
*   **500 Error:**
    *   The root cause is the division by zero in the `fail` function. You sho